In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st

# Load data


In [2]:
meat_csv_path = 'Resources/meatconsumption.csv'
meat_df = pd.read_csv(meat_csv_path)
meat_df.head(1)

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,MEATCONSUMP,BEEF,KG_CAP,A,1990,0.0,NaN


In [20]:
mortalitiy_csv_path = 'Resources/MortalityByUnintentionalPoisoning.csv'
mortality_df = pd.read_csv(mortalitiy_csv_path)
mortality_df.head(1)

,Indicator,Year,Geography,Sex,AgeGroup,Location,Value,Unit,Source,DataType
0,Mortality rate from unintentional poisoning (p...,2000,Afghanistan,Male,Not Available,Not Available,1.21,"Rate (per 100,000 population)",Global Health Observatory,NaN


In [6]:
meat_df.drop(['INDICATOR','Flag Codes','FREQUENCY'], axis=1)

,LOCATION,SUBJECT,MEASURE,TIME,Value
1280,ARG,BEEF,KG_CAP,1990,55.559
7920,ARG,BEEF,THND_TONNE,1990,2588.922
1281,ARG,BEEF,KG_CAP,1991,51.817
7921,ARG,BEEF,THND_TONNE,1991,2448.667
1282,ARG,BEEF,KG_CAP,1992,50.980
...,...,...,...,...,...
5677,ZAF,SHEEP,THND_TONNE,2027,182.702
4638,ZAF,SHEEP,KG_CAP,2028,2.554
5678,ZAF,SHEEP,THND_TONNE,2028,184.100
4639,ZAF,SHEEP,KG_CAP,2029,2.553


In [7]:

meat_df=meat_df.drop(['INDICATOR','Flag Codes','FREQUENCY'], axis=1)


In [9]:
meat_df = meat_df.rename(columns={"TIME":"Year"})

In [14]:
mortality_df=mortality_df.drop(['AgeGroup','Location','DataType'], axis=1)

In [16]:
mortality_df = mortality_df.rename(columns={"Geography":"LOCATION"})

In [18]:
merge_df = pd.merge(meat_df, mortality_df, on= ['Year','LOCATION'], how ="outer")